In [1]:
# Python black formater for notebook
%load_ext nb_black

<IPython.core.display.Javascript object>

# ETL
We are going to extract multiple data sources from the Dataset folder (csv & json), clean and transform those files into csv format to integrate and process into a Postgrestql database. 

In [56]:
# Importing modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import glob

<IPython.core.display.Javascript object>

## circuits.csv

In [3]:
circuits_df = pd.read_csv("Datasets/circuits.csv")

<IPython.core.display.Javascript object>

In [4]:
circuits_df.sample(10)

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
56,57,zeltweg,Zeltweg,Styria,Austria,47.2039,14.74780,676,http://en.wikipedia.org/wiki/Zeltweg_Airfield
75,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.9975,11.37190,255,http://en.wikipedia.org/wiki/Mugello_Circuit
5,6,monaco,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056,7,http://en.wikipedia.org/wiki/Circuit_de_Monaco
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.5700,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
22,23,osterreichring,A1-Ring,Spielburg,Austria,47.2197,14.76470,678,http://en.wikipedia.org/wiki/A1-Ring
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
7,8,magny_cours,Circuit de Nevers Magny-Cours,Magny Cours,France,46.8642,3.16361,228,http://en.wikipedia.org/wiki/Circuit_de_Nevers...
43,44,las_vegas,Las Vegas Street Circuit,Nevada,USA,36.1162,-115.17400,639,"http://en.wikipedia.org/wiki/Las_Vegas,_Nevada"
66,67,pedralbes,Circuit de Pedralbes,Barcelona,Spain,41.3903,2.11667,85,http://en.wikipedia.org/wiki/Pedralbes_Circuit
70,71,sochi,Sochi Autodrom,Sochi,Russia,43.4057,39.95780,2,http://en.wikipedia.org/wiki/Sochi_Autodrom


<IPython.core.display.Javascript object>

In [5]:
circuits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   circuitId   77 non-null     int64  
 1   circuitRef  77 non-null     object 
 2   name        77 non-null     object 
 3   location    77 non-null     object 
 4   country     77 non-null     object 
 5   lat         77 non-null     float64
 6   lng         77 non-null     float64
 7   alt         77 non-null     int64  
 8   url         77 non-null     object 
dtypes: float64(2), int64(2), object(5)
memory usage: 5.5+ KB


<IPython.core.display.Javascript object>

In [6]:
circuits_df.isnull().sum()

circuitId     0
circuitRef    0
name          0
location      0
country       0
lat           0
lng           0
alt           0
url           0
dtype: int64

<IPython.core.display.Javascript object>

In [7]:
circuits_df.to_csv("Datawarehouse/circuits.csv", index=False)

<IPython.core.display.Javascript object>

## constructors.json

In [8]:
constructors_df = pd.read_json("Datasets/constructors.json", lines=True)

<IPython.core.display.Javascript object>

In [9]:
constructors_df.sample(10)

,constructorId,constructorRef,name,nationality,url
18,19,jaguar,Jaguar,British,http://en.wikipedia.org/wiki/Jaguar_Racing
82,84,connew,Connew,British,http://en.wikipedia.org/wiki/Connew
75,77,trojan,Trojan,British,http://en.wikipedia.org/wiki/Trojan_(Racing_team)
33,34,brabham,Brabham,British,http://en.wikipedia.org/wiki/Brabham
199,202,shadow-ford,Shadow-Ford,British,http://en.wikipedia.org/wiki/Shadow_Racing_Cars
49,51,alfa,Alfa Romeo,Italian,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
94,96,derrington,Derrington,British,http://en.wikipedia.org/wiki/Derrington-Francis
138,140,klenk,Klenk,German,http://en.wikipedia.org/wiki/Klenk
103,105,maserati,Maserati,Italian,http://en.wikipedia.org/wiki/Maserati
139,141,simca,Simca,French,http://en.wikipedia.org/wiki/Simca


<IPython.core.display.Javascript object>

In [10]:
constructors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   constructorId   211 non-null    int64 
 1   constructorRef  211 non-null    object
 2   name            211 non-null    object
 3   nationality     211 non-null    object
 4   url             211 non-null    object
dtypes: int64(1), object(4)
memory usage: 8.4+ KB


<IPython.core.display.Javascript object>

In [11]:
constructors_df.isnull().sum()

constructorId     0
constructorRef    0
name              0
nationality       0
url               0
dtype: int64

<IPython.core.display.Javascript object>

In [12]:
constructors_df.to_csv("Datawarehouse/constructors.csv", index=False)

<IPython.core.display.Javascript object>

## drivers.json

In [13]:
with open("Datasets/drivers.json") as json_file:
    drivers_data = [json.loads(line) for line in json_file]

drivers_df = pd.json_normalize(drivers_data, sep="_")

<IPython.core.display.Javascript object>

In [14]:
drivers_df["number"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [15]:
drivers_df["code"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [16]:
drivers_df.sample(10)

,driverId,driverRef,number,code,dob,nationality,url,name_forename,name_surname
123,124,apicella,NaN,NaN,1965-10-07,Italian,http://en.wikipedia.org/wiki/Marco_Apicella,Marco,Apicella
279,280,pescarolo,NaN,NaN,1942-09-25,French,http://en.wikipedia.org/wiki/Henri_Pescarolo,Henri,Pescarolo
601,601,gibson,NaN,NaN,1918-04-16,British,http://en.wikipedia.org/wiki/Dick_Gibson,Dick,Gibson
447,448,broeker,NaN,NaN,1926-05-15,Canadian,http://en.wikipedia.org/wiki/Peter_Broeker,Peter,Broeker
160,161,weidler,NaN,NaN,1962-03-18,German,http://en.wikipedia.org/wiki/Volker_Weidler,Volker,Weidler
283,284,pesenti_rossi,NaN,NaN,1942-08-31,Italian,http://en.wikipedia.org/wiki/Alessandro_Pesent...,Alessandro,Pesenti-Rossi
704,704,cabantous,NaN,NaN,1904-10-08,French,http://en.wikipedia.org/wiki/Yves_Giraud_Caban...,Yves,Cabantous
222,223,regazzoni,NaN,NaN,1939-09-05,Swiss,http://en.wikipedia.org/wiki/Clay_Regazzoni,Clay,Regazzoni
285,286,brown,NaN,NaN,1949-12-24,Australian,http://en.wikipedia.org/wiki/Warwick_Brown,Warwick,Brown
194,195,guerrero,NaN,NaN,1958-11-16,Colombian,http://en.wikipedia.org/wiki/Roberto_Guerrero,Roberto,Guerrero


<IPython.core.display.Javascript object>

In [17]:
drivers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   driverId       853 non-null    int64  
 1   driverRef      853 non-null    object 
 2   number         47 non-null     float64
 3   code           96 non-null     object 
 4   dob            853 non-null    object 
 5   nationality    853 non-null    object 
 6   url            853 non-null    object 
 7   name_forename  853 non-null    object 
 8   name_surname   853 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 60.1+ KB


<IPython.core.display.Javascript object>

In [18]:
drivers_df.isnull().sum()

driverId           0
driverRef          0
number           806
code             757
dob                0
nationality        0
url                0
name_forename      0
name_surname       0
dtype: int64

<IPython.core.display.Javascript object>

In [19]:
drivers_df.to_csv("Datawarehouse/drivers.csv", index=False)

<IPython.core.display.Javascript object>

## pit_stops.json

In [20]:
pit_stops_df = pd.read_json("Datasets/pit_stops.json")

<IPython.core.display.Javascript object>

In [21]:
pit_stops_df.sample(10)

,raceId,driverId,stop,lap,time,duration,milliseconds
7471,1031,20,3,56,16:24:13,19.33,19330
709,852,3,1,11,14:25:16,23.09,23090
5633,970,826,4,6,14:18:28,17.322,17322
2264,883,821,2,16,15:32:19,21.997,21997
4840,953,4,2,32,14:56:10,26.836,26836
181,844,30,2,14,15:26:02,20.907,20907
693,851,1,6,56,15:30:48,13.173,13173
4870,954,831,2,35,14:51:16,23.608,23608
4744,951,825,1,16,15:34:05,30.585,30585
6946,1014,842,2,46,16:18:00,22.095,22095


<IPython.core.display.Javascript object>

In [22]:
pit_stops_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8030 entries, 0 to 8029
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   raceId        8030 non-null   int64 
 1   driverId      8030 non-null   int64 
 2   stop          8030 non-null   int64 
 3   lap           8030 non-null   int64 
 4   time          8030 non-null   object
 5   duration      8030 non-null   object
 6   milliseconds  8030 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 439.3+ KB


<IPython.core.display.Javascript object>

In [23]:
pit_stops_df["duration"] = pd.to_timedelta(
    pit_stops_df["milliseconds"],
    unit="ms",
)

<IPython.core.display.Javascript object>

In [24]:
pit_stops_df.sample(5)

,raceId,driverId,stop,lap,time,duration,milliseconds
385,847,67,2,20,13:37:27,0 days 00:00:24.649000,24649
1129,860,818,1,19,17:33:54,0 days 00:00:23.203000,23203
3618,914,18,2,9,15:44:19,0 days 00:00:24.532000,24532
5970,981,825,1,11,14:19:43,0 days 00:00:24.384000,24384
2982,899,823,1,24,14:36:01,0 days 00:00:23.732000,23732


<IPython.core.display.Javascript object>

In [25]:
pit_stops_df.to_csv("Datawarehouse/pit_stops.csv", index=False)

<IPython.core.display.Javascript object>

## races.csv

In [26]:
races_df = pd.read_csv("Datasets/races.csv")

<IPython.core.display.Javascript object>

In [27]:
races_df.sample(5)

,raceId,year,round,circuitId,name,date,time,url
524,525,1979,14,7,Canadian Grand Prix,1979-09-30,\N,http://en.wikipedia.org/wiki/1979_Canadian_Gra...
343,344,2010,8,7,Canadian Grand Prix,2010-06-13,16:00:00,http://en.wikipedia.org/wiki/2010_Canadian_Gra...
837,838,1950,6,55,French Grand Prix,1950-07-02,\N,http://en.wikipedia.org/wiki/1950_French_Grand...
189,190,1999,16,22,Japanese Grand Prix,1999-10-31,\N,http://en.wikipedia.org/wiki/1999_Japanese_Gra...
501,502,1980,5,40,Belgian Grand Prix,1980-05-04,\N,http://en.wikipedia.org/wiki/1980_Belgian_Gran...


<IPython.core.display.Javascript object>

In [28]:
races_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   raceId     1058 non-null   int64 
 1   year       1058 non-null   int64 
 2   round      1058 non-null   int64 
 3   circuitId  1058 non-null   int64 
 4   name       1058 non-null   object
 5   date       1058 non-null   object
 6   time       1058 non-null   object
 7   url        1058 non-null   object
dtypes: int64(4), object(4)
memory usage: 66.2+ KB


<IPython.core.display.Javascript object>

In [29]:
races_df.isnull().sum()

raceId       0
year         0
round        0
circuitId    0
name         0
date         0
time         0
url          0
dtype: int64

<IPython.core.display.Javascript object>

In [30]:
races_df["time"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [31]:
races_df.sample(5)

,raceId,year,round,circuitId,name,date,time,url
854,856,2011,16,35,Korean Grand Prix,2011-10-16,06:00:00,http://en.wikipedia.org/wiki/2011_Korean_Grand...
847,849,2011,9,9,British Grand Prix,2011-07-10,12:00:00,http://en.wikipedia.org/wiki/2011_British_Gran...
743,744,1961,7,14,Italian Grand Prix,1961-09-10,NaN,http://en.wikipedia.org/wiki/1961_Italian_Gran...
276,277,1993,5,4,Spanish Grand Prix,1993-05-09,NaN,http://en.wikipedia.org/wiki/1993_Spanish_Gran...
544,545,1977,3,30,South African Grand Prix,1977-03-05,NaN,http://en.wikipedia.org/wiki/1977_South_Africa...


<IPython.core.display.Javascript object>

In [32]:
races_df.to_csv("Datawarehouse/races.csv", index=False)

<IPython.core.display.Javascript object>

## results.json

In [33]:
with open("Datasets/results.json") as json_file:
    results_data = [json.loads(line) for line in json_file]

results_df = pd.json_normalize(results_data)

<IPython.core.display.Javascript object>

In [34]:
results_df.sample(5)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
13372,13373,548,200,1,2,9,4,4,4,3.0,76,+34.6,7107370,\N,\N,\N,\N,1
4528,4529,230,21,18,21,19,\N,R,17,0.0,1,\N,\N,\N,\N,\N,\N,4
15630,15631,632,278,82,19,2,5,5,5,2.0,78,\N,\N,\N,\N,\N,\N,11
5693,5694,277,104,18,23,20,8,8,8,0.0,63,\N,\N,\N,\N,\N,\N,12
19200,19201,798,648,131,20,3,\N,R,11,0.0,32,\N,\N,\N,\N,\N,\N,6


<IPython.core.display.Javascript object>

In [35]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24960 entries, 0 to 24959
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         24960 non-null  int64  
 1   raceId           24960 non-null  int64  
 2   driverId         24960 non-null  int64  
 3   constructorId    24960 non-null  int64  
 4   number           24960 non-null  object 
 5   grid             24960 non-null  int64  
 6   position         24960 non-null  object 
 7   positionText     24960 non-null  object 
 8   positionOrder    24960 non-null  int64  
 9   points           24960 non-null  float64
 10  laps             24960 non-null  int64  
 11  time             24960 non-null  object 
 12  milliseconds     24960 non-null  object 
 13  fastestLap       24960 non-null  object 
 14  rank             24960 non-null  object 
 15  fastestLapTime   24960 non-null  object 
 16  fastestLapSpeed  24960 non-null  object 
 17  statusId    

<IPython.core.display.Javascript object>

In [36]:
results_df[:].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [37]:
results_df.sample(5)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
16496,16497,673,358,191,4,5,NaN,R,9,0.0,55,NaN,NaN,NaN,NaN,NaN,NaN,95
774,775,54,24,5,20,13,11,11,11,0.0,54,NaN,NaN,40,16,1:37.387,204.902,12
20917,20920,847,18,1,4,7,1,1,1,25.0,70,4:04:39.537,14679537,69,1,1:16.956,204.007,1
8892,8893,384,131,44,33,0,NaN,F,29,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,81
10934,10935,461,186,50,17,0,NaN,F,27,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,81


<IPython.core.display.Javascript object>

In [38]:
results_df.isnull().sum()

resultId               0
raceId                 0
driverId               0
constructorId          0
number                 6
grid                   0
position           10735
positionText           0
positionOrder          0
points                 0
laps                   0
time               18368
milliseconds       18369
fastestLap         18410
rank               18249
fastestLapTime     18410
fastestLapSpeed    18410
statusId               0
dtype: int64

<IPython.core.display.Javascript object>

In [39]:
results_df.to_csv("Datawarehouse/results.csv", index=False)

<IPython.core.display.Javascript object>

## qualifying

### qualifying 1

In [40]:
qualifying_split_1_df = pd.read_json("Datasets/Qualifying/qualifying_split_1.json")

<IPython.core.display.Javascript object>

In [41]:
qualifying_split_1_df.sample(5)

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
116,117,23,17,9,10,9,1:16.074,1:15.745,1:17.343
2947,2949,271,94,18,23,16,1:39.548,\N,\N
513,514,42,3,3,16,14,1:13.128,1:13.060,\N
2462,2464,251,97,29,9,15,1:28.870,\N,\N
1299,1300,85,31,1,10,2,1:21.054,\N,\N


<IPython.core.display.Javascript object>

In [42]:
qualifying_split_1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qualifyId      4999 non-null   int64 
 1   raceId         4999 non-null   int64 
 2   driverId       4999 non-null   int64 
 3   constructorId  4999 non-null   int64 
 4   number         4999 non-null   int64 
 5   position       4999 non-null   int64 
 6   q1             4999 non-null   object
 7   q2             4999 non-null   object
 8   q3             4999 non-null   object
dtypes: int64(6), object(3)
memory usage: 351.6+ KB


<IPython.core.display.Javascript object>

In [43]:
qualifying_split_1_df["q2"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [44]:
qualifying_split_1_df["q3"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [45]:
qualifying_split_1_df.sample(5)

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
3760,3762,355,17,9,6,5,1:40.690,1:40.074,1:39.925
1863,1864,209,77,22,8,12,1:27.259,NaN,NaN
520,521,42,16,12,20,21,1:14.122,NaN,NaN
1183,1184,79,4,4,5,6,1:11.380,NaN,NaN
2503,2505,253,49,15,30,8,1:20.749,NaN,NaN


<IPython.core.display.Javascript object>

### qualifying 2

In [46]:
qualifying_split_2_df = pd.read_json("Datasets/Qualifying/qualifying_split_2.json")

<IPython.core.display.Javascript object>

In [47]:
qualifying_split_2_df.sample(5)

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
3653,8693,1045,154,210,8,19,1:30.138,\N,\N
326,5328,875,13,6,6,6,1:38.937,1:38.253,1:37.884
3472,8495,1036,847,3,63,18,1:18.099,\N,\N
49,5051,863,13,6,6,14,1:34.372,1:33.912,\N
3104,8127,1018,20,6,5,10,1:04.430,1:03.667,\N


<IPython.core.display.Javascript object>

In [48]:
qualifying_split_2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3695 entries, 0 to 3694
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qualifyId      3695 non-null   int64 
 1   raceId         3695 non-null   int64 
 2   driverId       3695 non-null   int64 
 3   constructorId  3695 non-null   int64 
 4   number         3695 non-null   int64 
 5   position       3695 non-null   int64 
 6   q1             3695 non-null   object
 7   q2             3695 non-null   object
 8   q3             3695 non-null   object
dtypes: int64(6), object(3)
memory usage: 259.9+ KB


<IPython.core.display.Javascript object>

In [49]:
qualifying_split_2_df["q2"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [50]:
qualifying_split_2_df["q3"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [51]:
qualifying_split_2_df.isnull().sum()

qualifyId           0
raceId              0
driverId            0
constructorId       0
number              0
position            0
q1                  0
q2                897
q3               1764
dtype: int64

<IPython.core.display.Javascript object>

In [52]:
qualifying_concat = pd.concat(
    [qualifying_split_1_df, qualifying_split_2_df], ignore_index=True
)

<IPython.core.display.Javascript object>

In [53]:
qualifying_concat

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...,...,...
8689,8730,1047,8,51,7,16,1:37.555,NaN,NaN
8690,8731,1047,825,210,20,17,1:37.863,NaN,NaN
8691,8732,1047,847,3,63,18,1:38.045,NaN,NaN
8692,8733,1047,850,210,51,19,1:38.173,NaN,NaN


<IPython.core.display.Javascript object>

In [54]:
qualifying_concat.to_csv("Datawarehouse/qualifying.csv", index=False)

<IPython.core.display.Javascript object>

## lap_times_split

In [57]:
all_files = glob.glob("Datasets/lap_times/*.csv")
lap_times_df = pd.concat(
    (
        pd.read_csv(
            f,
            header=None,
            names=["raceId", "driverId", "lab", "position", "time", "milliseconds"],
        )
        for f in all_files
    )
)

<IPython.core.display.Javascript object>

In [109]:
lap_times_df.sample(5)

,raceId,driverId,lab,position,time,milliseconds
96103,141,56,33,11,1:31.636,91636
88825,954,830,66,4,1:16.973,76973
72857,1031,815,62,3,1:08.748,68748
52264,1013,830,16,6,1:46.320,106320
22269,984,825,45,8,1:35.983,95983


<IPython.core.display.Javascript object>

In [110]:
lap_times_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490904 entries, 0 to 90904
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   raceId        490904 non-null  int64 
 1   driverId      490904 non-null  int64 
 2   lab           490904 non-null  int64 
 3   position      490904 non-null  int64 
 4   time          490904 non-null  object
 5   milliseconds  490904 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 26.2+ MB


<IPython.core.display.Javascript object>

In [111]:
lap_times_df.to_csv("Datawarehouse/lap_times.csv", index=False)

<IPython.core.display.Javascript object>